In [1]:
# torch 함수 불러오기
# onnx로 변환하기
# 변환된 onnx 검증하기

In [2]:
import numpy as np

# torch 함수 불러오기
import torch
from add_ai4 import Encoder
from add_ai4 import Decoder
from add_ai4 import Encoder_n_Decoder
# onnx로 변환하기
import torch.onnx
# 변환된 onnx 검증하기
import onnx # 검증만
import onnxruntime # 실제 계산까지

In [3]:
# 함수 불러오기
file_name = "data/add_ai4"

F = torch.load(file_name + '.pt', weights_only=False, map_location='cpu') # cpu로 불러오기
F.eval() # dropout 끄기

print(F)

Encoder_n_Decoder(
  (encoder): Encoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): LSTM(11, 11, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(12, 11, padding_idx=0)
    (rnn): LSTM(33, 11, batch_first=True, bidirectional=True)
    (f): Linear(in_features=44, out_features=12, bias=True)
  )
)


In [4]:
# onnx로 변환하기
x = torch.randint(0, 12, (1,7)).type(torch.long) # 입력값 더미 텐서
dynamic_axes = {'x' : {0 : 'b', 1 : 'f'}, 'y' : {0 : 'b'}} # 가변길이 차원 지정 dict

torch.onnx.export(
    F,
    x,
    file_name + '.onnx',
    export_params=True,
    opset_version=20,
    do_constant_folding=True,
    input_names=['x'],
    output_names=['y'],
    dynamic_axes=dynamic_axes
)

C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [5]:
# onnx 확인하기
onnx_F = onnx.load(file_name + '.onnx')

onnx.checker.check_model(onnx_F) #문제 없이 저장이 되었으면 오류 메세지 없음

In [6]:
# 실제 계산값 비교
onnx_F = onnxruntime.InferenceSession(file_name + '.onnx', providers=["CPUExecutionProvider"])
np_x = x.numpy()

y = F(x) # torch 계산 값

inputs = {onnx_F.get_inputs()[0].name : np_x}
outputs = onnx_F.run(None, inputs)
onnx_y = outputs[0] # onnx 계산 값

print(y.detach().numpy())
print(onnx_y)

np.testing.assert_allclose(y.detach().numpy(), onnx_y, rtol=1e-03, atol=1e-05) #계산 결과가 오차범위 내 차이 없으면 오류 메세지 없음

[[[-36.942387    31.429958     8.92841    -24.541452   -44.41225
   -42.131138   -21.36669     12.18445     32.549828    41.06372
    11.493965    -5.452112  ]
  [-10.928525    24.368797     9.098663   -18.043856   -32.33855
   -32.81287    -26.56531    -16.617216     2.483396    32.603428
    29.365519   -16.82774   ]
  [-10.379347    -3.0305548   -6.202652   -13.092377   -17.694893
   -19.53572    -16.12387    -11.606823    -1.4367698   14.426321
     6.8341856  -19.508898  ]
  [ 33.518322   -30.90601    -36.205044   -34.891937   -17.82804
    -0.27989483  10.262782    18.363842    23.23783     15.644077
    -9.762018   -13.82737   ]]]
[[[-36.9424      31.429985     8.92842    -24.541447   -44.41224
   -42.131134   -21.366692    12.1844425   32.549812    41.06371
    11.493982    -5.4521103 ]
  [-10.928526    24.368793     9.098658   -18.04386    -32.33856
   -32.812874   -26.56531    -16.61721      2.4834046   32.603428
    29.365517   -16.82774   ]
  [-10.379359    -3.0305667   -6.